In [5]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from tbparse import SummaryReader


def get_result(summary_files):
    df = None
    for sf in tqdm(summary_files):
        sr = SummaryReader(str(sf), pivot=True)
        d = pd.concat([sr.hparams, sr.scalars], axis=1)
        d.insert(0, 'name', int(list(sf.parents)[1].name))
        if df is None:
            df = d
        else:
            df = pd.concat([df,d], axis=0, ignore_index=True)
    return df.set_index('name')


In [7]:

dfs = []
scenes = ["ship_re", "lego_re", "ficus", "lyre"]
scenes = ["ship_re", "lego_re", "ficus"]

log_dir_pattern = "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/lv_rm_mlv"
log_dir_pattern = "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/{}/lv_rm_mlv_clamp_0_conv"

for s in scenes:
    log_dir = Path(log_dir_pattern.format(s))
    summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
    # summary_file = sorted(list(log_dir.glob("*/hparam_surf_close/*")))
    # summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))

    dfs.append(get_result(summary_file))

avg_df = dfs[0]

for i in range(1, len(dfs)):
    avg_df = avg_df.add(dfs[i])

avg_df = avg_df.dropna(axis=0, how='all') / len(dfs)

df_sorted = avg_df.sort_values(by='Chamfer/mean')[:30]

names = df_sorted.index.tolist()
print(str(names).replace(' ', ''))

df_sorted
# # df.sort_values(by='Image/psnr', ascending=False)[:30]
    

100%|██████████| 36/36 [00:01<00:00, 27.43it/s]

[1,12,7,5,6,4,0,16,13,17,24,2,28,11,18,25,10,14,3,19,29,9,8,21,20,22,23,15,32,33]


,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
1,0.0,3.0,10000.0,1.0,0.0,0.958904,0.942861,0.003236,0.950703,0.003094,0.002952,25.182676
12,0.0,5.0,5000.0,1.0,0.0,0.958780,0.942475,0.003269,0.950462,0.003111,0.002952,24.698632
7,0.0,3.0,40000.0,3.0,0.0,0.946885,0.950991,0.003583,0.948711,0.003120,0.002657,26.587732
5,0.0,3.0,10000.0,3.0,0.0,0.947114,0.950893,0.003577,0.948779,0.003120,0.002664,26.589551
6,0.0,3.0,25000.0,3.0,0.0,0.946831,0.950826,0.003584,0.948601,0.003122,0.002659,26.568541
4,0.0,3.0,5000.0,3.0,0.0,0.946762,0.950643,0.003586,0.948476,0.003125,0.002665,26.575479
0,0.0,3.0,5000.0,1.0,0.0,0.958543,0.940851,0.003287,0.949528,0.003126,0.002966,24.375172
16,0.0,5.0,5000.0,3.0,0.0,0.943890,0.950884,0.003650,0.947161,0.003160,0.002670,26.623863
13,0.0,5.0,10000.0,1.0,0.0,0.955269,0.942492,0.003318,0.948732,0.003165,0.003012,25.372225


In [8]:
name = [1]

df_row = dfs[0].loc[name]
for df in dfs[1:]:
    df_row = df_row.append(df.loc[name])

df_row

/tmp/ipykernel_14927/1663790657.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_row = df_row.append(df.loc[name])
/tmp/ipykernel_14927/1663790657.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_row = df_row.append(df.loc[name])


,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
1,0.0,3.0,10000.0,1.0,0,0.960651,0.954590,0.002853,0.957611,0.002877,0.002901,26.272039
1,0.0,3.0,10000.0,1.0,0,0.939909,0.895872,0.003911,0.917362,0.003917,0.003924,22.142778
1,0.0,3.0,10000.0,1.0,0,0.976153,0.978121,0.002945,0.977136,0.002489,0.002032,27.133211


In [38]:
name = [38, 50]

df_row = dfs[0].loc[name]
for df in dfs[1:]:
    df_row = df_row.append(df.loc[name])

df_row

/tmp/ipykernel_32478/568536522.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_row = df_row.append(df.loc[name])
/tmp/ipykernel_32478/568536522.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_row = df_row.append(df.loc[name])
/tmp/ipykernel_32478/568536522.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_row = df_row.append(df.loc[name])


,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
38,0.000001,3.0,25000.0,1.0,0,0.963366,0.952730,0.002688,0.958019,0.002767,0.002846,27.265463
50,0.000001,5.0,25000.0,1.0,0,0.962441,0.954563,0.002720,0.958486,0.002771,0.002822,27.355452
38,0.000001,3.0,25000.0,1.0,0,0.957766,0.876380,0.003169,0.915267,0.003665,0.004160,21.184952
50,0.000001,5.0,25000.0,1.0,0,0.956220,0.886301,0.003306,0.919934,0.003578,0.003850,21.584274
38,0.000001,3.0,25000.0,1.0,0,0.979396,0.982657,0.002775,0.981023,0.002223,0.001671,26.952854
50,0.000001,5.0,25000.0,1.0,0,0.978472,0.982629,0.002794,0.980546,0.002241,0.001688,27.256689
38,0.000001,3.0,25000.0,1.0,0,0.987917,0.996446,0.002036,0.992163,0.001675,0.001314,29.414976
50,0.000001,5.0,25000.0,1.0,0,0.984329,0.996239,0.002186,0.990248,0.001791,0.001396,29.394335


In [25]:
df.loc[
    (df['lr_alpha']==1e-2) & \
    (df['lr_alpha_final']==0.0001) &\
    # (df['lr_alpha_final']==1e-4) & \
    # (df['lr_alpha_final']==1e-4) & \
    (df['surf_init_alpha_rescale']==0.1)
    
    ].sort_values(by='Chamfer/mean')[:30]

,name,lr_alpha,lr_alpha_final,lr_surface,lr_surface_final,surf_init_alpha_rescale,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
10,0010,0.01,0.0001,0.0010,1.000000e-07,0.1,0,0.902140,0.948974,0.004195,0.924965,0.003872,0.003550,21.421875
13,0013,0.01,0.0001,0.0010,1.000000e-06,0.1,0,0.898561,0.951430,0.004259,0.924240,0.003878,0.003496,21.570852
7,0007,0.01,0.0001,0.0001,1.000000e-05,0.1,0,0.905243,0.945102,0.004128,0.924743,0.003880,0.003633,21.408058
16,0016,0.01,0.0001,0.0010,1.000000e-05,0.1,0,0.889970,0.953837,0.004489,0.920797,0.004018,0.003548,21.486448
4,0004,0.01,0.0001,0.0001,1.000000e-06,0.1,0,0.904947,0.938096,0.004215,0.921223,0.004045,0.003874,21.079668
1,0001,0.01,0.0001,0.0001,1.000000e-07,0.1,0,0.903727,0.933666,0.004279,0.918452,0.004155,0.004031,20.986023


In [2]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/lv_rm_mlv_sparsity")
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/lv_rm_mlv_clamp_0_conv")
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/lv_erm_mlv")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_surf_close/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))

df3 = get_result(summary_file)

df_sorted = df3.sort_values(by='Chamfer/mean')[:30]

names = df_sorted.index.tolist()

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████| 36/36 [00:02<00:00, 14.23it/s]

[1,5,7,6,4,12,13,16,17,2,0,28,18,11,19,24,10,3,29,9,14,25,8,20,21,22,23,32,15,33]


,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
1,0.0,3.0,10000.0,1.0,0,0.960651,0.954590,0.002853,0.957611,0.002877,0.002901,26.272039
5,0.0,3.0,10000.0,3.0,0,0.946919,0.966932,0.003223,0.956821,0.002893,0.002563,27.495356
7,0.0,3.0,40000.0,3.0,0,0.946711,0.966994,0.003230,0.956745,0.002896,0.002561,27.487761
6,0.0,3.0,25000.0,3.0,0,0.946411,0.966909,0.003236,0.956550,0.002897,0.002558,27.450689
4,0.0,3.0,5000.0,3.0,0,0.946364,0.966256,0.003239,0.956206,0.002904,0.002569,27.451147
12,0.0,5.0,5000.0,1.0,0,0.959300,0.951207,0.002896,0.955237,0.002928,0.002960,25.701136
13,0.0,5.0,10000.0,1.0,0,0.955864,0.954459,0.002957,0.955161,0.002942,0.002927,26.799555
16,0.0,5.0,5000.0,3.0,0,0.943638,0.966410,0.003306,0.954888,0.002942,0.002578,27.554506
17,0.0,5.0,10000.0,3.0,0,0.941647,0.966369,0.003375,0.953848,0.002969,0.002564,27.502186


In [14]:
df3.loc[[95]]

,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
95,0.00001,5.0,40000.0,5.0,0,0.862824,0.851956,0.005698,0.857356,0.005937,0.006176,25.817764


In [4]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/lv_rm_mlv_sparsity")
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/lv_rm_mlv_clamp_0_conv")
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/lv_erm_mlv")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_surf_close/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))

df4 = get_result(summary_file)

df_sorted = df4.sort_values(by='Chamfer/mean')[:30]

names = df_sorted.index.tolist()

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████| 36/36 [00:02<00:00, 13.26it/s]

[1,12,0,13,24,2,5,7,6,4,16,14,25,17,3,28,11,18,10,19,29,9,8,15,21,23,20,22,32,26]


,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
1,0.0,3.0,10000.0,1.0,0,0.976153,0.978121,0.002945,0.977136,0.002489,0.002032,27.133211
12,0.0,5.0,5000.0,1.0,0,0.975565,0.976772,0.002975,0.976168,0.002526,0.002076,26.116926
0,0.0,3.0,5000.0,1.0,0,0.975626,0.976124,0.002977,0.975875,0.002530,0.002082,25.431421
13,0.0,5.0,10000.0,1.0,0,0.974149,0.977432,0.003009,0.975788,0.002547,0.002086,27.217127
24,0.0,10.0,5000.0,1.0,0,0.973547,0.973361,0.003051,0.973454,0.002613,0.002176,24.911995
2,0.0,3.0,25000.0,1.0,0,0.969735,0.977105,0.003133,0.973406,0.002624,0.002115,27.232454
5,0.0,3.0,10000.0,3.0,0,0.958704,0.986669,0.003607,0.972485,0.002701,0.001794,27.761000
7,0.0,3.0,40000.0,3.0,0,0.958293,0.986865,0.003618,0.972369,0.002701,0.001784,27.774963
6,0.0,3.0,25000.0,3.0,0,0.958350,0.986573,0.003616,0.972257,0.002703,0.001790,27.757860


In [18]:
df2.loc[[85, 95]]

,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
85,0.00001,5.0,10000.0,1.0,0,0.958320,0.927500,0.002732,0.942658,0.003378,0.004023,27.226788
95,0.00001,5.0,40000.0,5.0,0,0.862871,0.852063,0.005697,0.857433,0.005935,0.006173,25.816591


In [13]:
df2.loc[
    # (df2['lr_alpha']==1e-2) & \
    # (df2['lr_alpha_final']==0.0001) &\
    (df2['truncated_vol_render_a_final']==5)
    
    ].sort_values(by='Chamfer/mean')[:30]

,lambda_conv_mode_samp,truncated_vol_render_a,truncated_vol_render_a_decay_steps,truncated_vol_render_a_final,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,,
35,1.000000e-07,10.0,40000.0,5.0,0,0.922861,0.948287,0.003665,0.935401,0.003367,0.003069,28.272095
20,1.000000e-07,5.0,5000.0,5.0,0,0.912964,0.955805,0.004016,0.933893,0.003437,0.002857,28.379398
23,1.000000e-07,5.0,40000.0,5.0,0,0.912868,0.955721,0.004021,0.933803,0.003440,0.002858,28.379417
22,1.000000e-07,5.0,25000.0,5.0,0,0.912759,0.955550,0.004020,0.933665,0.003441,0.002862,28.376995
21,1.000000e-07,5.0,10000.0,5.0,0,0.912728,0.955796,0.004026,0.933766,0.003443,0.002860,28.379114
32,1.000000e-07,10.0,5000.0,5.0,0,0.907720,0.956026,0.004161,0.931247,0.003503,0.002844,28.417725
33,1.000000e-07,10.0,10000.0,5.0,0,0.902522,0.955949,0.004294,0.928467,0.003569,0.002845,28.439087
8,1.000000e-07,3.0,5000.0,5.0,0,0.928737,0.937488,0.003584,0.933092,0.003592,0.003599,29.010866
9,1.000000e-07,3.0,10000.0,5.0,0,0.929830,0.936560,0.003562,0.933183,0.003594,0.003626,29.022053


In [6]:
# SummaryReader
df = None
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/single_lv_low_prune")
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/mlv_trunc_l1_sparse")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
summary_file = sorted(list(log_dir.glob("*/hparam_surf_close/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))

df3 = get_result(summary_file)

df_sorted = df3.sort_values(by='Chamfer/mean')[:30]

names = df_sorted.index.tolist()

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████| 56/56 [00:02<00:00, 20.42it/s]

[19,5,33,47,17,18,31,32,30,46,45,4,28,29,3,42,43,44,16,15,14,2,1,0,6,48,34,20,22,50]


,alpha_surf_sparsify_sparsity,lambda_conv_mode_samp,lambda_sparsify_alpha,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,
19,1.0,1.000000e-06,1.000000e-09,0,0.957122,0.930202,0.002674,0.943470,0.003285,0.003896,25.119736
5,1.0,1.000000e-08,1.000000e-09,0,0.956116,0.931861,0.002707,0.943833,0.003296,0.003885,25.092194
33,1.0,5.000000e-06,1.000000e-09,0,0.956487,0.925157,0.002686,0.940561,0.003340,0.003995,25.097353
47,1.0,1.000000e-05,1.000000e-09,0,0.955430,0.921437,0.002724,0.938125,0.003407,0.004089,25.078344
17,1.0,1.000000e-06,1.000000e-10,0,0.947372,0.930748,0.003028,0.938986,0.003455,0.003882,27.934547
18,0.1,1.000000e-06,1.000000e-09,0,0.947517,0.930858,0.003029,0.939114,0.003455,0.003880,27.924988
31,1.0,5.000000e-06,1.000000e-10,0,0.950113,0.924194,0.002922,0.936974,0.003491,0.004060,27.944252
32,0.1,5.000000e-06,1.000000e-09,0,0.950221,0.924399,0.002920,0.937132,0.003491,0.004062,27.933521
30,0.1,5.000000e-06,1.000000e-10,0,0.948424,0.924001,0.003048,0.936053,0.003560,0.004072,28.016993


In [10]:
# SummaryReader
df = None
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/single_lv_low_prune")
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/lego_re/mlv_trunc_l1_sparse")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_surf_close/*")))

df3 = get_result(summary_file)

df_sorted = df3.sort_values(by='Chamfer/mean')[:30]

names = df_sorted.index.tolist()

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████| 40/40 [00:01<00:00, 25.31it/s]

[18,17,16,14,15,4,3,2,1,0,32,31,46,30,45,28,29,44,42,43,19,5,33,47,48,34,6,20,50,36]


,alpha_surf_sparsify_sparsity,lambda_conv_mode_samp,lambda_sparsify_alpha,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,
18,0.1,1.000000e-06,1.000000e-09,0,0.911339,0.828850,0.004257,0.868139,0.004972,0.005687,24.426041
17,1.0,1.000000e-06,1.000000e-10,0,0.910117,0.828980,0.004298,0.867656,0.004992,0.005687,24.549307
16,0.1,1.000000e-06,1.000000e-10,0,0.898194,0.829662,0.004516,0.862569,0.005024,0.005532,26.792849
14,0.1,1.000000e-06,0.000000e+00,0,0.897232,0.829341,0.004535,0.861952,0.005033,0.005531,27.059601
15,1.0,1.000000e-06,0.000000e+00,0,0.897100,0.829445,0.004539,0.861947,0.005035,0.005530,27.057165
4,0.1,1.000000e-08,1.000000e-09,0,0.914266,0.833327,0.004342,0.871922,0.005047,0.005751,24.469837
3,1.0,1.000000e-08,1.000000e-10,0,0.912986,0.832958,0.004388,0.871138,0.005072,0.005756,24.594326
2,0.1,1.000000e-08,1.000000e-10,0,0.901795,0.835047,0.004577,0.867139,0.005086,0.005595,26.872654
1,1.0,1.000000e-08,0.000000e+00,0,0.900957,0.835116,0.004588,0.866788,0.005088,0.005588,27.135851


In [8]:
df3.loc[[19]]

,alpha_surf_sparsify_sparsity,lambda_conv_mode_samp,lambda_sparsify_alpha,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,,
19,1.0,0.000001,1.000000e-09,0,0.896294,0.818264,0.005084,0.855503,0.005821,0.006558,17.573702


In [21]:
# SummaryReader
df = None
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/single_lv_low_prune")
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/lego_re/mlv_trunc_conv_2")
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/mlv_trunc_tv")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = sorted(list(log_dir.glob("*/hparam_surf_close/*")))

df3 = get_result(summary_file)

df_sorted = df3.sort_values(by='Chamfer/mean')[:30]

names = df_sorted.index.tolist()

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████| 18/18 [00:00<00:00, 22.21it/s]

[4,5,3,1,2,11,9,8,10,7,15,17,14,16,18,0,19,12]


,lambda_normal_loss,lambda_tv_surface,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
name,,,,,,,,,,
4,0.00000,1.000000e-04,0,0.969887,0.982963,0.002964,0.976381,0.002482,0.001999,24.027233
5,0.00000,1.000000e-03,0,0.984898,0.966990,0.002549,0.975862,0.002529,0.002508,24.204060
3,0.00000,1.000000e-05,0,0.962059,0.984958,0.003199,0.973374,0.002579,0.001959,23.841658
1,0.00000,1.000000e-08,0,0.961167,0.985505,0.003229,0.973184,0.002589,0.001949,23.800369
2,0.00000,1.000000e-06,0,0.961495,0.985200,0.003228,0.973203,0.002590,0.001951,23.815336
11,0.00001,1.000000e-04,0,0.965085,0.981004,0.003191,0.972979,0.002626,0.002061,23.943079
9,0.00001,1.000000e-06,0,0.963492,0.983179,0.003246,0.973236,0.002629,0.002011,23.832788
8,0.00001,1.000000e-08,0,0.963281,0.982905,0.003248,0.972994,0.002633,0.002018,23.835941
10,0.00001,1.000000e-05,0,0.963057,0.982869,0.003253,0.972862,0.002635,0.002017,23.836189


In [14]:
names = list({0,12,5,4,19,11,1,2,3,7,8,10,9,5,12,7,8,10,19,14,17,15,16,18})
str(names).replace(' ', '')

'[0,1,2,3,4,5,7,8,9,10,11,12,14,15,16,17,18,19]'

In [ ]:
df.iloc[[76]]

In [21]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/rew_init")
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/lego_re/slow_surf_mlv_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = [sorted(list(f.glob('*')))[0] for f in summary_folder]
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df is None:
        df = d
    else:
        df = pd.concat([df,d], axis=0, ignore_index=True)

df_sorted = df.sort_values(by='Chamfer/mean')[:30]

names = df_sorted['name'].values.tolist()
names = list(map(int, names))

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████| 243/243 [00:15<00:00, 15.87it/s]

[9,0,1,10,27,117,90,99,108,81,111,84,102,93,120,82,91,118,100,109,126,127,3,94,85,103,112,121,129,18]


,name,lr_alpha,lr_alpha_final,lr_surface,lr_surface_final,surf_init_density_lvs,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
9,0009,0.001,0.00010,0.000001,1.000000e-10,['10'],0,0.958357,0.991261,0.003387,0.974532,0.002549,0.001710,26.584822
0,0000,0.001,0.00001,0.000001,1.000000e-10,['10'],0,0.958355,0.991261,0.003387,0.974530,0.002549,0.001710,26.427746
1,0001,0.001,0.00001,0.000001,1.000000e-08,['10'],0,0.958311,0.991263,0.003397,0.974509,0.002555,0.001713,26.596460
10,0010,0.001,0.00010,0.000001,1.000000e-08,['10'],0,0.958278,0.991254,0.003400,0.974487,0.002556,0.001713,26.778296
27,0027,0.010,0.00001,0.000001,1.000000e-10,['10'],0,0.957417,0.991525,0.003447,0.974173,0.002577,0.001708,27.046410
117,0117,0.010,0.00010,0.000001,1.000000e-10,['25'],0,0.954878,0.990938,0.003466,0.972574,0.002588,0.001711,25.875921
90,0090,0.001,0.00010,0.000001,1.000000e-10,['25'],0,0.954873,0.990938,0.003466,0.972571,0.002588,0.001711,25.749874
99,0099,0.001,0.00100,0.000001,1.000000e-10,['25'],0,0.954871,0.990938,0.003466,0.972570,0.002588,0.001711,25.810497
108,0108,0.010,0.00001,0.000001,1.000000e-10,['25'],0,0.954876,0.990938,0.003466,0.972573,0.002588,0.001711,25.799482
81,0081,0.001,0.00001,0.000001,1.000000e-10,['25'],0,0.954872,0.990938,0.003466,0.972571,0.002588,0.001711,25.734818


In [6]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/slow_surf_mlv_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = [sorted(list(f.glob('*')))[0] for f in summary_folder]
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df is None:
        df = d
    else:
        df = pd.concat([df,d], axis=0, ignore_index=True)

df_sorted = df.sort_values(by='Chamfer/mean')

names = df_sorted['name'].values.tolist()
names = list(map(int, names))

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:01<00:00, 20.11it/s]

[50,50,51,21,20,290,291,81,80,80,321,29,281,26,26,280,22,24,268,271,270,254,311,310,111,110,110,256,256,244,258,301,300,246,299]


,name,lambda_l_entropy,lambda_normal_loss,lr_surface,lr_surface_final,no_surf_norm_con_check,surf_init_alpha_rescale,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
7,0050,0.01000,0.000000e+00,0.00001,1.000000e-06,1.0,0.1,0,0.961690,0.991128,0.003301,0.976187,0.002495,0.001689,25.397461
8,0050,0.01000,0.000000e+00,0.00001,1.000000e-06,1.0,0.1,0,0.961676,0.991140,0.003301,0.976186,0.002495,0.001690,25.397287
9,0051,0.01000,0.000000e+00,0.00001,1.000000e-06,0.0,0.1,0,0.961675,0.991153,0.003302,0.976191,0.002495,0.001689,25.397429
1,0021,0.01000,0.000000e+00,0.00001,1.000000e-08,0.0,0.1,0,0.960194,0.991449,0.003339,0.975571,0.002511,0.001682,25.227085
0,0020,0.01000,0.000000e+00,0.00001,1.000000e-08,1.0,0.1,0,0.960182,0.991455,0.003340,0.975568,0.002511,0.001682,25.227472
27,0290,0.01000,0.000000e+00,0.00001,1.000000e-06,1.0,1.0,0,0.959161,0.991564,0.003383,0.975093,0.002543,0.001702,24.916121
28,0291,0.01000,0.000000e+00,0.00001,1.000000e-06,0.0,1.0,0,0.959150,0.991562,0.003385,0.975087,0.002543,0.001702,24.916082
12,0081,0.01000,0.000000e+00,0.00010,1.000000e-08,0.0,0.1,0,0.961323,0.990707,0.003340,0.975794,0.002551,0.001762,25.647573
10,0080,0.01000,0.000000e+00,0.00010,1.000000e-08,1.0,0.1,0,0.961271,0.990764,0.003342,0.975795,0.002552,0.001761,25.646961
11,0080,0.01000,0.000000e+00,0.00010,1.000000e-08,1.0,0.1,0,0.961240,0.990735,0.003343,0.975765,0.002552,0.001761,25.647163


In [10]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/vase/slow_surf_mlv_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = [sorted(list(f.glob('*')))[0] for f in summary_folder]
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df is None:
        df = d
    else:
        df = pd.concat([df,d], axis=0, ignore_index=True)

df_sorted = df.sort_values(by='Chamfer/mean')[:30]

names = df_sorted['name'].values.tolist()
names = list(map(int, names))

print(str(names).replace(' ', ''))

df_sorted
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:02<00:00, 13.35it/s]

[254,256,300,301,270,271,281,280,246,311,244,291,290,258,268,321,22,24,26,21,20,50,51,29,80,81,110,111]


,name,lambda_l_entropy,lambda_normal_loss,lr_surface,lr_surface_final,no_surf_norm_con_check,surf_init_alpha_rescale,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
14,0254,0.00100,1.000000e-06,0.00001,1.000000e-08,1.0,1.0,0,0.708550,0.774585,0.010716,0.740097,0.008280,0.005845,29.133139
15,0256,0.00100,1.000000e-05,0.00001,1.000000e-08,1.0,1.0,0,0.708470,0.773660,0.010699,0.739631,0.008282,0.005865,29.137527
24,0300,0.00001,0.000000e+00,0.00010,1.000000e-08,1.0,1.0,0,0.705555,0.773651,0.010741,0.738036,0.008307,0.005874,29.162275
25,0301,0.00001,0.000000e+00,0.00010,1.000000e-08,0.0,1.0,0,0.705459,0.773502,0.010742,0.737916,0.008309,0.005876,29.162281
18,0270,0.00001,0.000000e+00,0.00001,1.000000e-06,1.0,1.0,0,0.707539,0.770247,0.010681,0.737563,0.008311,0.005941,29.155704
19,0271,0.00001,0.000000e+00,0.00001,1.000000e-06,0.0,1.0,0,0.707569,0.770189,0.010682,0.737552,0.008312,0.005942,29.155781
21,0281,0.00100,0.000000e+00,0.00001,1.000000e-06,0.0,1.0,0,0.708426,0.768433,0.010636,0.737211,0.008315,0.005994,29.161852
20,0280,0.00100,0.000000e+00,0.00001,1.000000e-06,1.0,1.0,0,0.708406,0.768417,0.010635,0.737192,0.008316,0.005996,29.161911
13,0246,0.00001,1.000000e-05,0.00001,1.000000e-08,1.0,1.0,0,0.705053,0.777343,0.010835,0.739435,0.008317,0.005800,29.133434
26,0311,0.00100,0.000000e+00,0.00010,1.000000e-08,0.0,1.0,0,0.707514,0.769272,0.010669,0.737101,0.008320,0.005970,29.166267


In [7]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/slow_surf_mlv_2")
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ship_re/slow_surf_mlv_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = [sorted(list(f.glob('*')))[0] for f in summary_folder]
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df is None:
        df = d
    else:
        df = pd.concat([df,d], axis=0, ignore_index=True)

df.sort_values(by='Chamfer/mean')[:30]
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476/476 [00:27<00:00, 17.08it/s]


,name,lambda_l_entropy,lambda_normal_loss,lr_surface,lr_surface_final,no_surf_norm_con_check,surf_init_alpha_rescale,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
76,0080,0.01000,0.000000e+00,0.00010,1.000000e-08,1.0,0.1,0,0.935208,0.974673,0.003341,0.954533,0.002884,0.002427,27.365114
77,0081,0.01000,0.000000e+00,0.00010,1.000000e-08,0.0,0.1,0,0.935192,0.974783,0.003341,0.954577,0.002885,0.002428,27.365278
267,0271,0.00001,0.000000e+00,0.00001,1.000000e-06,0.0,1.0,0,0.932985,0.973527,0.003412,0.952825,0.002893,0.002374,26.918709
266,0270,0.00001,0.000000e+00,0.00001,1.000000e-06,1.0,1.0,0,0.932889,0.973578,0.003417,0.952799,0.002894,0.002372,26.919113
46,0050,0.01000,0.000000e+00,0.00001,1.000000e-06,1.0,0.1,0,0.934864,0.969904,0.003333,0.952061,0.002900,0.002467,27.157772
47,0051,0.01000,0.000000e+00,0.00001,1.000000e-06,0.0,0.1,0,0.934806,0.969884,0.003335,0.952022,0.002902,0.002470,27.157148
106,0110,0.01000,0.000000e+00,0.00010,1.000000e-06,1.0,0.1,0,0.933703,0.976912,0.003401,0.954819,0.002917,0.002433,27.607176
264,0268,0.01000,1.000000e-04,0.00001,1.000000e-08,1.0,1.0,0,0.937525,0.967415,0.003349,0.952235,0.002919,0.002490,26.827816
107,0111,0.01000,0.000000e+00,0.00010,1.000000e-06,0.0,0.1,0,0.933367,0.976756,0.003407,0.954569,0.002921,0.002434,27.607031
277,0281,0.00100,0.000000e+00,0.00001,1.000000e-06,0.0,1.0,0,0.934611,0.968410,0.003357,0.951211,0.002922,0.002487,26.877771


In [8]:
df.loc[
    (df['lambda_l_entropy']==0.00100) & \
    (df['lambda_normal_loss']==1e-5) & \
    (df['lr_surface']==0.00001) & \
    (df['lr_surface_final']==1e-8) & \
    (df['no_surf_norm_con_check']==1)
    ]

,name,lambda_l_entropy,lambda_normal_loss,lr_surface,lr_surface_final,no_surf_norm_con_check,surf_init_alpha_rescale,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
16,0016,0.001,0.00001,0.00001,1.000000e-08,1.0,0.1,0,0.866758,0.989318,0.005479,0.923992,0.003857,0.002234,28.603870
252,0256,0.001,0.00001,0.00001,1.000000e-08,1.0,1.0,0,0.931982,0.972077,0.003464,0.951607,0.002933,0.002403,26.791895


In [18]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/slow_surf_mlv_2")
# log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/lego_re/slow_surf_mlv_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
summary_file = sorted(list(log_dir.glob("*/hparam_mode01/*")))
# summary_file = [sorted(list(f.glob('*')))[0] for f in summary_folder]
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df is None:
        df = d
    else:
        df = pd.concat([df,d], axis=0, ignore_index=True)

df.sort_values(by='Chamfer/mean')[:30]
# df.sort_values(by='Image/psnr', ascending=False)[:30]
# df.iloc[[24]]

0it [00:00, ?it/s]


AttributeError: 'NoneType' object has no attribute 'sort_values'

In [21]:
df2.loc[
    (df2['alpha_lv_sets']==50.0) & \
    # (df2['lambda_inplace_alpha_sparsify']==1e-8) & \
    (df2['lambda_l_entropy']==5e-3) & \
    (df2['lr_alpha']==0.1) & \
    (df2['lr_surface']==1e-3)
    ]

,name,alpha_lv_sets,lambda_inplace_alpha_sparsify,lambda_l_entropy,lr_alpha,lr_surface,step,Chamfer/accuracy,Chamfer/completeness,Chamfer/d2s,Chamfer/f1,Chamfer/mean,Chamfer/s2d,Image/psnr
147,0147,50.0,1.000000e-10,0.005,0.1,0.001,0,0.730128,0.962888,0.020744,0.830507,0.012004,0.003265,28.318964
155,0155,50.0,1.000000e-09,0.005,0.1,0.001,0,0.798759,0.962949,0.014561,0.873203,0.008918,0.003275,28.259121
163,0163,50.0,1.000000e-08,0.005,0.1,0.001,0,0.944741,0.964979,0.004179,0.954753,0.003772,0.003365,28.154858
171,0171,50.0,5.000000e-08,0.005,0.1,0.001,0,0.959163,0.959322,0.003473,0.959243,0.003592,0.003711,27.637205
179,0179,50.0,1.000000e-07,0.005,0.1,0.001,0,0.955640,0.946611,0.003790,0.951104,0.003971,0.004152,27.052227
187,0187,50.0,5.000000e-07,0.005,0.1,0.001,0,0.926539,0.855756,0.005160,0.889742,0.005567,0.005974,24.471239


In [26]:
# SummaryReader
df = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/inplace_as_l1_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
# summary_folder = sorted(list(log_dir.glob("*/hparam_mesh/")))
# summary_file = [sorted(list(f.glob('*')))[0] for f in summary_folder]
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df is None:
        df = d
    else:
        df = pd.concat([df,d], axis=0, ignore_index=True)

df.sort_values(by='Chamfer/mean')[:30]
# df.iloc[[24]]

100%|██████████| 12/12 [00:00<00:00, 45.51it/s]


,name,lambda_inplace_alpha_sparsify,lambda_l_entropy,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
9,0009,1.000000e-06,0.02,0,0.003605,0.003673,0.003741
5,0005,1.000000e-05,0.01,0,0.003409,0.003700,0.003991
4,0004,5.000000e-06,0.01,0,0.003656,0.003725,0.003794
10,0010,5.000000e-06,0.02,0,0.003660,0.003864,0.004068
8,0008,5.000000e-07,0.02,0,0.004146,0.003933,0.003719
3,0003,1.000000e-06,0.01,0,0.004475,0.003965,0.003454
11,0011,1.000000e-05,0.02,0,0.003575,0.004002,0.004429
6,0006,1.000000e-08,0.02,0,0.005139,0.004421,0.003702
7,0007,1.000000e-07,0.02,0,0.005229,0.004479,0.003729
2,0002,5.000000e-07,0.01,0,0.006517,0.004936,0.003354


In [12]:
df.iloc[[14,15,16,17,18]]

,name,lambda_l_dist,lambda_l_entropy,lambda_l_samp_dist,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
14,0015,0.0,0.01,0.0000,0,0.005926,0.005633,0.005340
15,0016,0.0,0.01,0.0001,0,0.005790,0.005573,0.005356
16,0017,0.0,0.01,0.0010,0,0.005736,0.005633,0.005530
17,0018,0.0,0.01,0.0100,0,0.006181,0.006787,0.007394
18,0019,0.0,0.01,0.1000,0,0.007158,0.007172,0.007187


In [5]:
df.loc[
    # (df['lambda_normal_loss']==1.0) & \
    # (df['lambda_sparsify_alpha']==1e-20) & \
    (df['lambda_sparsify_surf']==1e-6) & \
    (df['sparsify_surf_thresh']==0.15)
    ]

,lambda_sparsify_alpha,lambda_sparsify_surf,sparsify_surf_thresh,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
7,0.00001,0.000001,0.15,0,0.032569,0.019676,0.006784
32,0.00010,0.000001,0.15,0,0.031542,0.019176,0.006810
57,0.00100,0.000001,0.15,0,0.029813,0.018146,0.006478
82,0.00500,0.000001,0.15,0,0.026745,0.016489,0.006233
107,0.01000,0.000001,0.15,0,0.025097,0.015654,0.006212
132,0.05000,0.000001,0.15,0,0.022323,0.014380,0.006437


In [11]:
# SummaryReader
df2 = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/l_entropy_2")

summary_file = sorted(list(log_dir.glob("*/hparam_pts/*")))
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    d.insert(0, 'name', list(sf.parents)[1].name)
    if df2 is None:
        df2 = d
    else:
        df2 = pd.concat([df2,d], axis=0, ignore_index=True)

df2.sort_values(by='Chamfer/mean')[:30]

100%|██████████| 32/32 [00:01<00:00, 20.51it/s]


,name,lambda_l_dist,lambda_l_entropy,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
27,0027,0.00100,0.00100,0,0.003156,0.003420,0.003684
25,0025,0.00100,0.00001,0,0.003210,0.003421,0.003631
24,0024,0.00100,0.00000,0,0.003326,0.003454,0.003582
26,0026,0.00100,0.00010,0,0.003759,0.003694,0.003630
28,0028,0.00100,0.01000,0,0.003387,0.003718,0.004049
20,0020,0.00010,0.01000,0,0.004436,0.003967,0.003499
5,0005,0.00000,0.10000,0,0.003999,0.004797,0.005594
13,0013,0.00001,0.10000,0,0.004332,0.004946,0.005559
21,0021,0.00010,0.10000,0,0.004320,0.005001,0.005682
29,0029,0.00100,0.10000,0,0.003850,0.005164,0.006479


In [8]:
df2.iloc[[4]]

,name,lambda_l_dist,lambda_l_entropy,lambda_sparsify_alpha,lambda_sparsify_surf,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
4,0004,0.0,0.01,0.0,0.0,0,0.010391,0.006837,0.003283


In [38]:
df3.loc[
    (df['lambda_normal_loss']==0.001) & \
    # (df['lambda_sparsify_alpha']==0.001) & \
    # (df['lambda_sparsify_surf']==0.0001	) & \
    (df['lambda_tv_alpha']==0.001	)
    ]

,lambda_normal_loss,lambda_sparsify_alpha,lambda_sparsify_surf,lambda_tv_alpha,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
4,0.001,1.000000e-08,1.000000e-08,0.001,0,0.003763,0.003212,0.002661
10,0.001,1.000000e-08,1.000000e-06,0.001,0,0.003766,0.003113,0.002460
16,0.001,1.000000e-08,1.000000e-04,0.001,0,0.007990,0.005463,0.002936
22,0.001,1.000000e-05,1.000000e-08,0.001,0,0.003721,0.003197,0.002674
28,0.001,1.000000e-05,1.000000e-06,0.001,0,0.003799,0.003131,0.002463
34,0.001,1.000000e-05,1.000000e-04,0.001,0,0.007831,0.005365,0.002898
40,0.001,1.000000e-03,1.000000e-08,0.001,0,0.004188,0.003482,0.002776
46,0.001,1.000000e-03,1.000000e-06,0.001,0,0.005891,0.004251,0.002611
52,0.001,1.000000e-03,1.000000e-04,0.001,0,0.007505,0.005085,0.002666
58,0.001,1.000000e-02,1.000000e-08,0.001,0,0.006505,0.004731,0.002956


In [39]:
df2.loc[
    (df['lambda_normal_loss']==0.001) & \
    # (df['lambda_sparsify_alpha']==0.001) & \
    # (df['lambda_sparsify_surf']==0.0001	) & \
    (df['lambda_tv_alpha']==0.001	)
    ]

,lambda_normal_loss,lambda_sparsify_alpha,lambda_sparsify_surf,lambda_tv_alpha,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
4,0.001,1.000000e-08,1.000000e-08,0.001,0,0.035021,0.020319,0.005616
10,0.001,1.000000e-08,1.000000e-06,0.001,0,0.032463,0.019062,0.005661
16,0.001,1.000000e-08,1.000000e-04,0.001,0,0.031905,0.018683,0.005461
22,0.001,1.000000e-05,1.000000e-08,0.001,0,0.035077,0.020367,0.005656
28,0.001,1.000000e-05,1.000000e-06,0.001,0,0.032503,0.019140,0.005776
34,0.001,1.000000e-05,1.000000e-04,0.001,0,0.027250,0.016362,0.005474
40,0.001,1.000000e-03,1.000000e-08,0.001,0,0.032095,0.018894,0.005693
46,0.001,1.000000e-03,1.000000e-06,0.001,0,0.030632,0.018214,0.005795
52,0.001,1.000000e-03,1.000000e-04,0.001,0,0.021097,0.013031,0.004966
58,0.001,1.000000e-02,1.000000e-08,0.001,0,0.027460,0.016314,0.005168


In [2]:
# SummaryReader
df3 = None
log_dir = Path("/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/auto_tune/ficus/reg_general")

summary_file = sorted(list(log_dir.glob("*/hparam/*")))
for sf in tqdm(summary_file):
    sr = SummaryReader(str(sf), pivot=True)
    d = pd.concat([sr.hparams, sr.scalars], axis=1)
    if df3 is None:
        df3 = d
    else:
        df3 = pd.concat([df3,d], axis=0, ignore_index=True)

df3.sort_values(by='Chamfer/mean')[:30]

100%|██████████| 72/72 [00:05<00:00, 12.27it/s]


,lambda_normal_loss,lambda_sparsify_alpha,lambda_sparsify_surf,lambda_tv_alpha,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
8,0.0100,1.000000e-08,1.000000e-06,0.00001,0,0.003326,0.003007,0.002687
29,0.0100,1.000000e-05,1.000000e-06,0.00100,0,0.003338,0.003017,0.002695
26,0.0100,1.000000e-05,1.000000e-06,0.00001,0,0.003311,0.003032,0.002753
7,0.0010,1.000000e-08,1.000000e-06,0.00001,0,0.003625,0.003048,0.002472
11,0.0100,1.000000e-08,1.000000e-06,0.00100,0,0.003498,0.003104,0.002710
10,0.0010,1.000000e-08,1.000000e-06,0.00100,0,0.003766,0.003113,0.002460
28,0.0010,1.000000e-05,1.000000e-06,0.00100,0,0.003799,0.003131,0.002463
20,0.0100,1.000000e-05,1.000000e-08,0.00001,0,0.003338,0.003152,0.002965
41,0.0100,1.000000e-03,1.000000e-08,0.00100,0,0.003266,0.003157,0.003047
38,0.0100,1.000000e-03,1.000000e-08,0.00001,0,0.003297,0.003160,0.003023


In [18]:
df2.iloc[df3.sort_values(by='Chamfer/mean')[:50].index]

,lambda_normal_loss,lambda_sparsify_alpha,lambda_sparsify_surf,lambda_tv_alpha,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
8,0.0100,1.000000e-08,1.000000e-06,0.00001,0,0.032184,0.019376,0.006568
29,0.0100,1.000000e-05,1.000000e-06,0.00100,0,0.031671,0.019217,0.006763
26,0.0100,1.000000e-05,1.000000e-06,0.00001,0,0.032031,0.019427,0.006823
7,0.0010,1.000000e-08,1.000000e-06,0.00001,0,0.032576,0.019193,0.005811
11,0.0100,1.000000e-08,1.000000e-06,0.00100,0,0.031421,0.018968,0.006515
10,0.0010,1.000000e-08,1.000000e-06,0.00100,0,0.032463,0.019062,0.005661
28,0.0010,1.000000e-05,1.000000e-06,0.00100,0,0.032503,0.019140,0.005776
20,0.0100,1.000000e-05,1.000000e-08,0.00001,0,0.034193,0.020364,0.006535
41,0.0100,1.000000e-03,1.000000e-08,0.00100,0,0.032601,0.019608,0.006614
38,0.0100,1.000000e-03,1.000000e-08,0.00001,0,0.032289,0.019410,0.006531


In [20]:
df2.iloc[[52,8]]

,lambda_normal_loss,lambda_sparsify_alpha,lambda_sparsify_surf,lambda_tv_alpha,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
52,0.001,1.000000e-03,0.000100,0.00100,0,0.021097,0.013031,0.004966
8,0.010,1.000000e-08,0.000001,0.00001,0,0.032184,0.019376,0.006568


In [12]:
# df.loc[(df['fake_sample_std']==1.0) & (df['fake_sample_std_final']==0.01) & (df['lr_alpha_final']==0.05) & (df['surf_init_alpha_rescale']==0.01)]
df2.loc[
    (df['fake_sample_std']==1.0) & \
    (df['fake_sample_std_final']==0.01) & \
    (df['lr_alpha']==1.0) & \
    # (df['lr_alpha_final']==0.01) & \
    (df['surf_init_alpha_rescale']==0.01)
    ]

,fake_sample_std,fake_sample_std_final,lr_alpha,lr_alpha_final,surf_init_alpha_rescale,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
150,1.0,0.01,1.0,0.0001,0.01,0,0.003963,0.003222,0.002481
158,1.0,0.01,1.0,0.0010,0.01,0,0.004024,0.003263,0.002502
166,1.0,0.01,1.0,0.0100,0.01,0,0.005328,0.004000,0.002671
174,1.0,0.01,1.0,0.0500,0.01,0,0.007867,0.005472,0.003077
182,1.0,0.01,1.0,0.1000,0.01,0,0.009128,0.006236,0.003344
190,1.0,0.01,1.0,0.5000,0.01,0,0.011015,0.007529,0.004042


In [18]:
df2.iloc[166]

fake_sample_std            1.000000
fake_sample_std_final      0.010000
lr_alpha                   1.000000
lr_alpha_final             0.010000
surf_init_alpha_rescale    0.010000
step                       0.000000
Chamfer/d2s                0.005328
Chamfer/mean               0.004000
Chamfer/s2d                0.002671
Name: 166, dtype: float64

In [5]:

cf_file = sorted(list(log_dir.glob("*/ckpt_eval_*/ckpt/cf.txt")))
cf_file

cf = np.zeros([len(cf_file), 3])

for i, cf_f in enumerate(cf_file):
    with cf_f.open() as f:
        cf[i, 0] = float(f.readline().split(' ')[-1])
        cf[i, 1] = float(f.readline().split(' ')[-1])
        cf[i, 2] = float(f.readline().split(' ')[-1])

df = pd.DataFrame(cf, columns=['d2s', 's2d', 'mean'])
df.sort_values(by='mean')[:30]

,d2s,s2d,mean
656,0.003772,0.002818,0.003295
658,0.003788,0.002820,0.003304
659,0.003790,0.002825,0.003307
657,0.003800,0.002824,0.003312
582,0.003887,0.002840,0.003363
580,0.003886,0.002850,0.003368
640,0.003835,0.002902,0.003368
581,0.003905,0.002839,0.003372
583,0.003911,0.002843,0.003377
654,0.003901,0.002854,0.003377


In [29]:
df.sort_values(by='Chamfer/mean')[:30]

,lambda_normal_loss,lr_alpha,lr_alpha_final,lr_surface,lr_surface_final,step,Chamfer/d2s,Chamfer/mean,Chamfer/s2d
656,0.0001,0.10,0.00100,0.0001,1.000000e-05,0,0.003772,0.003295,0.002818
658,0.0100,0.10,0.00100,0.0001,1.000000e-05,0,0.003788,0.003304,0.002820
659,0.1000,0.10,0.00100,0.0001,1.000000e-05,0,0.003790,0.003307,0.002825
657,0.0010,0.10,0.00100,0.0001,1.000000e-05,0,0.003800,0.003312,0.002824
582,0.0100,0.01,0.00100,0.0001,1.000000e-06,0,0.003887,0.003363,0.002840
580,0.0001,0.01,0.00100,0.0001,1.000000e-06,0,0.003886,0.003368,0.002850
640,0.0001,0.01,0.00100,0.0001,1.000000e-05,0,0.003835,0.003368,0.002902
581,0.0010,0.01,0.00100,0.0001,1.000000e-06,0,0.003905,0.003372,0.002839
583,0.1000,0.01,0.00100,0.0001,1.000000e-06,0,0.003911,0.003377,0.002843
654,0.0100,0.10,0.00010,0.0001,1.000000e-05,0,0.003901,0.003377,0.002854
